In [ ]:
pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 7.9 MB/s 
     |████████████████████████████████| 840 kB 62.8 MB/s 
     |████████████████████████████████| 120 kB 68.7 MB/s 
     |████████████████████████████████| 1.1 MB 68.4 MB/s 
     |████████████████████████████████| 6.3 MB 44.2 MB/s 
     |████████████████████████████████| 3.4 MB 54.5 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 213 kB 44.6 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 52.8 MB/s 
     |████████████████████████████████| 1.1 MB 32.0 MB/s 
     |████████████████████████████████| 1.2 MB 43.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 99 kB 6.0 MB/s 
     |████████████████████████████████| 47.6 MB 74 kB/s 
     |████████████████████████████████| 352 kB 55.9 MB/s 
     |████████████████████████████████| 208 kB 44.6 MB/s 


In [ ]:
import pandas as pd
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.models import save_model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import time
import imutils
from imutils.video import VideoStream
import cv2
import numpy as np
import os


from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
image_path = list(paths.list_images('')) # Se carga el directorio del dataset
                  
full_data =[] # Vector para recolectar imagenes
etiquetas = [] # Vector para recolectar las etiquetas
for data_path in image_path:
    etiqueta = data_path.split(os.path.sep)[-2] #Se obtiene la etiqueta de cada imagen.
    image = img_to_array(load_img(data_path, target_size=(224, 224)))
    image = preprocess_input(image) #Convierte la imagen a un formato especifico para Movilnet V2
    full_data.append(image) #Agrega la imagen
    etiquetas.append(etiqueta) # Agrega la etiqueta
data =np.array(full_data,dtype='float32') # Se convierte las imagenes a numpy
etiquetas= np.array(etiquetas)  # Se convierte las etiquetas a numpy
lb = LabelBinarizer()
raw_label = etiquetas
etiquetas = lb.fit_transform(etiquetas)
etiquetas = to_categorical(etiquetas) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
(x_train, x_test, y_train, y_test) = train_test_split(data, etiquetas, test_size=0.20, stratify=etiquetas, random_state=250)
# Se divide la data en 80% de entrenamiento y 20% para testeo.

In [ ]:
Main_model = MobileNetV2(weights="imagenet", include_top=False,  # Preperar el MovilNet V2
                         input_tensor=Input(shape=(224, 224, 3)))

Main_out_put = Main_model.output

#Creación de la capa de salida
head_model = AveragePooling2D(pool_size = (7,7))(Main_out_put)
Flatten_layer = Flatten(name="flatten")(head_model)
Dense_layer = Dense(128, activation="relu")(Flatten_layer)
Dropouts = Dropout(0.5)(Dense_layer)
Output_layer = Dense(2, activation="softmax")(Dropouts)


model = Model(inputs = Main_model.input, outputs = Output_layer)

for layer in Main_model.layers: #Congelar las capas del main model para que no se actualicen en la primera corrida 
    layer.trainable = False

In [31]:
class Optimizer:
    def __init__(self, model, mb = 8, lr = 0.0001, loss = tf.keras.losses.binary_crossentropy, 
               opt=tf.keras.optimizers.Adam, regularization = "l1",lamda = 0.01):
        self.model     = model 
        self.loss      = loss 
        self.optimizer = opt(learning_rate = lr) 
        self.mb        = mb  
        self.l1_l2_regul = self.regularization_type(regularization)
        self.reg_const = lamda
        self.train_loss     = tf.keras.metrics.Mean()
        self.train_accuracy = tf.keras.metrics.CategoricalAccuracy()
        self.test_loss     = tf.keras.metrics.Mean()
        self.test_accuracy = tf.keras.metrics.CategoricalAccuracy()

   
    def train_step(self, x , y):
        with tf.GradientTape() as tape:
            predictions = model(x) 
            loss_temp = self.loss(y, predictions) 
            loss = self.apply_reg(loss_temp) 

        #Applying gradients
        gradients = tape.gradient(loss, self.model.trainable_variables) 
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.train_loss(loss) 
        self.train_accuracy(y, predictions) 
        return loss

    
    def test_step(self, x , y):
        predictions = self.model(x) 
        loss = self.loss(y, predictions) 
        self.test_loss(loss) 
        self.test_accuracy(y, predictions) 
  
    def apply_reg(self,dyn_loss): 
        reg_loss = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES) 
        regularizer = self.l1_l2_regul(self.reg_const)(reg_loss) 
        loss = tf.reduce_mean(dyn_loss + regularizer)
        return loss


    def regularization_type(self,querry):
        if querry == "l1":
            return regularizers.l1
        elif querry == "l2":
            return regularizers.l2


    def train (self):
        batches =0
        for mbX, mbY in self.Augumentation.flow(self.trainX,self.trainY, batch_size = self.mb):
            self.train_step(mbX, mbY)
            batches +=1
            if batches >= len(x_train) / 32:
                break
  

    def test  (self):
        batches = 0
        for mbX, mbY in self.Augumentation.flow(self.testX,self.testY, batch_size = self.mb):
            self.test_step(mbX, mbY)
            batches +=1
            if batches >= len(x_train) / 32:
                break
  

    def run   (self, dataX, dataY, testX, testY, epochs, verbose=2):
        historyTR = [] 
        historyTS = [] 
        historyTR_acc = [] 
        historyTS_acc = [] 
        template = '{} {}, {}: {}, {}: {}, {}: {},{}: {}' 
                                                

        self.Augumentation = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,
                                   height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")
        self.Augumentation.fit(x_train)


        self.trainX = dataX
        self.trainY = dataY 
        self.testX  = testX
        self.testY = testY 

        for i in range(epochs):
      
            self.train () 
            self.test  () 
            if verbose > 0:
                print(template.format("epoch: ", i+1," TRAIN LOSS: ", self.train_loss.result(),
                              " TEST LOSS: " , self.test_loss.result(),
                              " TRAIN ACC: " , self.train_accuracy.result()*100,
                              " TEST ACC: "  , self.test_accuracy.result()*100))
            temp = '{}'
            historyTR.append(float(temp.format(self.train_loss.result())))
            historyTS.append(float(temp.format(self.test_loss.result() )))
            historyTR_acc.append(float(temp.format(self.train_accuracy.result()*100)))
            historyTS_acc.append(float(temp.format(self.test_accuracy.result()*100 )))
            self.train_loss.reset_states()
            self.train_accuracy.reset_states()
            self.test_loss.reset_states()
            self.test_accuracy.reset_states()
        
        return historyTR, historyTS,historyTR_acc,historyTS_acc

In [ ]:
opt    = Optimizer (model, mb = 20, lr = 0.0001,regularization = "l1",lamda = 0.01 ) 
epochs=10
loss,val_loss,acc,val_acc = opt.run (x_train, y_train, x_test, y_test, epochs, verbose=1)

In [ ]:
model.save('/content/drive/MyDrive/Mascarillas 4/my_model.h5')


In [ ]:
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predIdxs = model.predict(x_test, batch_size=32)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,target_names=lb.classes_))

